In [1]:
!pip install transformers --upgrade -q
!pip install datasets -q
!pip install accelerate -U -q

In [2]:
import tensorflow as tf
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
import json

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("Wikidepia/indobert-lite-squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# Prepare Train Set

In [43]:
df = pd.read_json("/content/data_processed.json")

In [44]:
df['question'] = df['question'].str.lower()
df['context'] = df['context'].str.lower()
df['answer'] = df['answer'].str.lower()

In [45]:
df['answer_start'] = df.apply(lambda row: row["context"].find(row["answer"]), axis=1)

In [46]:
df.at[2, 'answer_start'] = 0
df.at[3, 'answer_start'] = 10
df.at[12, 'answer_start'] = 0
df.at[19, 'answer_start'] = 0
df.at[29, 'answer_start'] = 0
df.at[31, 'answer_start'] = 4

In [47]:
df['answer'] = df['answer'].apply(lambda x: json.dumps({'text': [x], 'answer_start': int(df['answer_start'][0])}))

In [48]:
questions = [q.strip() for q in df["question"]]
context = [q.strip() for q in df["context"]]
inputs = tokenizer(
        questions,
        context,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

offset_mapping = inputs.pop("offset_mapping")

start_positions = []
end_positions = []
answers = df['answer'].apply(lambda x: json.loads(x))
for i, offset in enumerate(offset_mapping):
    answer = answers[i]
    start_char = int(answer["answer_start"])
    end_char = start_char + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label it (0, 0)
    if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

df["start_positions"] = start_positions
df["end_positions"] = end_positions

In [50]:
import pandas as pd
from datasets import Dataset

data = {
    'input_ids': inputs['input_ids'],
    'attention_mask': inputs['attention_mask'],
    'start_positions': start_positions,
    'end_positions': end_positions,
    'answer': answers,
}

# Create DataFrame
df = pd.DataFrame(data)
df.to_csv('encoding_train.csv', index=False)
train = Dataset.from_pandas(df)

In [51]:
df

,input_ids,attention_mask,start_positions,end_positions,answer
0,"[2, 916, 2132, 2705, 3102, 477, 29955, 3, 2132...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",12,17,{'text': ['secara offline melalui computer bas...
1,"[2, 387, 377, 2976, 2705, 3102, 26, 13595, 818...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",19,44,"{'text': ['seleksi mandiri umum (cbt), seleksi..."
2,"[2, 2609, 1162, 2789, 2705, 3102, 26, 13595, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",17,30,{'text': ['biaya pendaftaran ujian mandiri di ...
3,"[2, 2609, 1162, 1171, 29836, 90, 2976, 3102, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",16,49,{'text': ['ukt untuk jalur ujian mandiri diken...
4,"[2, 1262, 1617, 1092, 11049, 2419, 29846, 2976...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",17,36,{'text': ['melalui laman https://damba.uinsgd....
5,"[2, 937, 176, 2976, 857, 396, 11408, 18860, 64...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",28,48,{'text': ['terdapat jalur tahfidz bagi hafidz ...
6,"[2, 387, 377, 4794, 2705, 3102, 3749, 10831, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",16,103,{'text': ['rangkaian tes yang harus diikuti ca...
7,"[2, 387, 4794, 2976, 3848, 26, 13595, 8185, 23...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",18,62,{'text': ['penerimaan mahasiswa baru jalur pre...
8,"[2, 387, 4794, 2976, 3848, 302, 29947, 4703, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",22,42,{'text': ['calon mahasiswa baru diharuskan mel...
9,"[2, 916, 4368, 2789, 1744, 440, 90, 2976, 2967...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",21,50,"{'text': ['tatacara pendaftaran, jadwal seleks..."


# Build Model

In [52]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator
from sklearn.model_selection import KFold
from transformers import DefaultDataCollator

In [53]:
# Define the model
model = AutoModelForQuestionAnswering.from_pretrained("Wikidepia/indobert-lite-squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [73]:
# Define the training arguments with early stopping
training_args = TrainingArguments(
    output_dir="./output_dir",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir='./logs',
    logging_steps=10,
)


## Train using K-Fold

In [75]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and evaluation sets
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

# Create Datasets from DataFrames
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

# Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()

print("Evaluation results:", eval_results)

Epoch,Training Loss,Validation Loss
1,No log,5.368585
2,0.183600,5.503879
3,0.217400,5.572516


Evaluation results: {'eval_loss': 5.572515964508057, 'eval_runtime': 0.213, 'eval_samples_per_second': 42.254, 'eval_steps_per_second': 14.085, 'epoch': 3.0}


## Evaluation

## Save Model

In [78]:
model_path = "models"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('your_model_directory/tokenizer_config.json',
 'your_model_directory/special_tokens_map.json',
 'your_model_directory/vocab.txt',
 'your_model_directory/added_tokens.json',
 'your_model_directory/tokenizer.json')

## Make Predictions

In [80]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer


# 2. Load the saved model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 3. Prepare your input data
context = "Pelaksanaan Ujian Mandiri dilaksanakan secara offline melalui Computer Based Test (CBT)."
question = "Bagaimana pelaksanaan Ujian Mandiri dilakukan ?"

# Tokenize the input
inputs = tokenizer(question, context, padding="max_length", truncation="only_second", max_length=384, return_tensors="pt")

# 4. Perform inference using the loaded model
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted start and end indices
start_index = torch.argmax(outputs.start_logits, dim=-1).item()
end_index = torch.argmax(outputs.end_logits, dim=-1).item()

# Convert token indices to text span
all_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
answer_tokens = all_tokens[start_index:end_index+1]
answer = tokenizer.convert_tokens_to_string(answer_tokens)

print("Predicted Answer:", answer)

Predicted Answer: secara offline melalui computer based test
